In [175]:
import numpy as np
from numpy import ndarray

In [176]:
import plotly.graph_objects as go
def save_to_html(fig, path, file_name):
    # Comment this line if you get an error
    fig.write_html(path + '/' + file_name)

In [177]:
x = np.array([-1, -1e-3, 1e-3, 1])

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y= np.heaviside(x,0.5), name='heaviside'))
fig.update_layout(width=600, yaxis_range=[-0.1, 1.1], title='heaviside function')
fig.show()

In [178]:
save_to_html(fig, '../Slides/ANN', 'fig1.html')

In [179]:
x = np.hstack( (np.linspace(-4,-1e-3,32), np.linspace(1e-3,4,32)) )

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y= np.heaviside(x,0.5), name='heaviside'))
fig.add_trace(go.Scatter(x=x, y= 1/(1 + np.exp(-x)), name='sigmoid'))
fig.add_trace(go.Scatter(x=x, y= np.tanh(x), name='tanh'))
fig.add_trace(go.Scatter(x=x, y= np.maximum(x, 0), name='relu'))
fig.update_layout(width=600, yaxis_range=[-1.1, 2.1])
fig.show()

In [180]:
x = np.hstack( (np.linspace(-4,-1e-3,32), np.linspace(1e-3,4,32)) )

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y= 1/(1 + np.exp(-x)), name='sigmoid', line_width=8))
fig.update_layout(width=450, yaxis_range=[-0.5, 1.5])
fig.show()

In [181]:
save_to_html(fig, '../Slides/ANN', 'fig2.html')

In [182]:
def fun(x):
    return 1. / (1. + x**2)

X_train = np.linspace(-1,1,1025)
y_train = fun(X_train)

X_valid = np.linspace(-1,1,257)
y_valid = fun(X_valid)

X_test = np.linspace(-1,1,129)
y_test = fun(X_test)

In [183]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train, y=y_train))
fig.update_layout(width=600,title='Function to approximate', xaxis_title='x')
fig.show()

In [184]:
save_to_html(fig, '../Slides/TF_Keras', 'fig1.html')

In [185]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

n_epochs = 100
learning_rate = 0.1

In [186]:
tf.random.set_seed(2020)
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=1))
model.add(keras.layers.Dense(4, activation="tanh"))
model.add(keras.layers.Dense(4, activation="tanh"))
model.add(keras.layers.Dense(4, activation="tanh"))
model.add(keras.layers.Dense(1, activation="linear"))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 4)                 8         
_________________________________________________________________
dense_53 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_54 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_55 (Dense)             (None, 1)                 5         
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________


In [187]:
model.layers

In [188]:
hidden1 = model.layers[1]
print(hidden1.name)

dense_53


In [189]:
weights, biases = hidden1.get_weights()
print(weights.shape)
print(biases.shape)

(4, 4)
(4,)


In [190]:
sgd = optimizers.SGD(lr=learning_rate)
model.compile(loss='mse', optimizer=sgd, metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Train on 1025 samples, validate on 257 samples
Epoch 1/100
1025/1025 [==============================] - 2s 1ms/sample - loss: 0.0295 - mse: 0.0295 - mae: 0.1045 - val_loss: 0.0158 - val_mse: 0.0158 - val_mae: 0.1051
Epoch 2/100
1025/1025 [==============================] - 0s 297us/sample - loss: 0.0040 - mse: 0.0040 - mae: 0.0517 - val_loss: 0.0026 - val_mse: 0.0026 - val_mae: 0.0405
Epoch 3/100
1025/1025 [==============================] - 0s 286us/sample - loss: 0.0017 - mse: 0.0017 - mae: 0.0320 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0317
Epoch 4/100
1025/1025 [==============================] - 0s 136us/sample - loss: 7.3423e-04 - mse: 7.3423e-04 - mae: 0.0203 - val_loss: 5.2598e-04 - val_mse: 5.2598e-04 - val_mae: 0.0173
Epoch 5/100
1025/1025 [==============================] - 0s 300us/sample - loss: 4.4904e-04 - mse: 4.4904e-04 - mae: 0.0162 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0337
Epoch 6/100
1025/1025 [==============================] - 0s 193us/sample - l

Epoch 45/100
1025/1025 [==============================] - 0s 168us/sample - loss: 2.0302e-05 - mse: 2.0302e-05 - mae: 0.0037 - val_loss: 2.0569e-05 - val_mse: 2.0569e-05 - val_mae: 0.0037
Epoch 46/100
1025/1025 [==============================] - 0s 141us/sample - loss: 2.0259e-05 - mse: 2.0259e-05 - mae: 0.0037 - val_loss: 3.5271e-05 - val_mse: 3.5271e-05 - val_mae: 0.0049
Epoch 47/100
1025/1025 [==============================] - 0s 134us/sample - loss: 2.0542e-05 - mse: 2.0542e-05 - mae: 0.0037 - val_loss: 2.9903e-05 - val_mse: 2.9903e-05 - val_mae: 0.0045
Epoch 48/100
1025/1025 [==============================] - 0s 111us/sample - loss: 1.9455e-05 - mse: 1.9455e-05 - mae: 0.0036 - val_loss: 1.8218e-05 - val_mse: 1.8218e-05 - val_mae: 0.0035
Epoch 49/100
1025/1025 [==============================] - 0s 118us/sample - loss: 1.8136e-05 - mse: 1.8136e-05 - mae: 0.0034 - val_loss: 2.7430e-05 - val_mse: 2.7430e-05 - val_mae: 0.0042
Epoch 50/100
1025/1025 [==============================] - 0s

1025/1025 [==============================] - 0s 119us/sample - loss: 7.6928e-06 - mse: 7.6928e-06 - mae: 0.0020 - val_loss: 1.3010e-05 - val_mse: 1.3010e-05 - val_mae: 0.0030
Epoch 89/100
1025/1025 [==============================] - 0s 192us/sample - loss: 6.8585e-06 - mse: 6.8585e-06 - mae: 0.0019 - val_loss: 1.3382e-05 - val_mse: 1.3382e-05 - val_mae: 0.0031
Epoch 90/100
1025/1025 [==============================] - 0s 118us/sample - loss: 6.6810e-06 - mse: 6.6810e-06 - mae: 0.0019 - val_loss: 7.2425e-06 - val_mse: 7.2425e-06 - val_mae: 0.0019
Epoch 91/100
1025/1025 [==============================] - 0s 140us/sample - loss: 6.2215e-06 - mse: 6.2215e-06 - mae: 0.0018 - val_loss: 9.4738e-06 - val_mse: 9.4738e-06 - val_mae: 0.0025
Epoch 92/100
1025/1025 [==============================] - 0s 117us/sample - loss: 6.1510e-06 - mse: 6.1510e-06 - mae: 0.0018 - val_loss: 7.7469e-06 - val_mse: 7.7469e-06 - val_mae: 0.0021
Epoch 93/100
1025/1025 [==============================] - 0s 114us/sample

In [191]:
def plot_history(fig, history):
    error = history.history["mse"]
    error_val = history.history["val_mse"]
    fig.add_trace(go.Scatter(y=error,name='error training'))
    fig.add_trace(go.Scatter(y=error_val, name='error validation'))
    fig.update_layout(width=600, yaxis_type="log", 
                      xaxis_range=[0,100], yaxis_range=[-6,0],
                      xaxis_title='epoch')    

fig = go.Figure()
plot_history(fig, history)
fig.show()

In [192]:
save_to_html(fig, '../Slides/TF_Keras', 'fig2.html')

In [193]:
model.evaluate(X_test, y_test)

129/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.0624886346154516e-05, 1.0624885e-05, 0.0026588861]

In [194]:
y_pred = np.array( model.predict(X_test) ).flatten()
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_test, y=y_test, name='exact'))
fig.add_trace(go.Scatter(x=X_test, y=y_pred, name='model'))
fig.update_layout(width=600, xaxis_title='x')
fig.show()

In [195]:
save_to_html(fig, '../Slides/TF_Keras', 'fig3.html')

In [196]:
def plot_error(fig, model, X_test, y_test):
    y_pred = np.array( model.predict(X_test) ).flatten()
    fig.add_trace(go.Scatter(x = X_test, y = abs(y_test-y_pred)))
    fig.update_layout(width=600, yaxis_type='log', title='absolute error vs x',
                     xaxis_range=[-1,1], yaxis_range=[-6,-1], xaxis_title='x')

fig = go.Figure()
plot_error(fig, model, X_test, y_test)
fig.show()

In [197]:
save_to_html(fig, '../Slides/TF_Keras', 'fig4.html')

In [198]:
from tensorflow.keras import layers, Model

In [199]:
tf.random.set_seed(2020)
input_ = layers.Input(shape=1)
hidden1 = layers.Dense(4, activation="tanh")(input_)
hidden2 = layers.Dense(4, activation="tanh")(hidden1)
hidden3 = layers.Dense(4, activation="tanh")(hidden2)
concat = layers.Concatenate()([hidden1, hidden2, hidden3])
output = layers.Dense(1, activation="linear")(concat)
model = Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_56 (Dense)                (None, 4)            8           input_13[0][0]                   
__________________________________________________________________________________________________
dense_57 (Dense)                (None, 4)            20          dense_56[0][0]                   
__________________________________________________________________________________________________
dense_58 (Dense)                (None, 4)            20          dense_57[0][0]                   
____________________________________________________________________________________________

In [200]:
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Train on 1025 samples, validate on 257 samples
Epoch 1/100
1025/1025 [==============================] - 1s 1ms/sample - loss: 0.0995 - mse: 0.0995 - mae: 0.1852 - val_loss: 0.0270 - val_mse: 0.0270 - val_mae: 0.1296
Epoch 2/100
1025/1025 [==============================] - 0s 292us/sample - loss: 0.0128 - mse: 0.0128 - mae: 0.0995 - val_loss: 0.0116 - val_mse: 0.0116 - val_mae: 0.0931
Epoch 3/100
1025/1025 [==============================] - 0s 164us/sample - loss: 0.0093 - mse: 0.0093 - mae: 0.0851 - val_loss: 0.0145 - val_mse: 0.0145 - val_mae: 0.1056
Epoch 4/100
1025/1025 [==============================] - 0s 164us/sample - loss: 0.0064 - mse: 0.0064 - mae: 0.0702 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0675
Epoch 5/100
1025/1025 [==============================] - 0s 144us/sample - loss: 0.0038 - mse: 0.0038 - mae: 0.0540 - val_loss: 0.0032 - val_mse: 0.0032 - val_mae: 0.0485
Epoch 6/100
1025/1025 [==============================] - 0s 166us/sample - loss: 0.0021 - mse: 0.002

Epoch 45/100
1025/1025 [==============================] - 0s 140us/sample - loss: 1.1216e-05 - mse: 1.1216e-05 - mae: 0.0028 - val_loss: 1.1919e-05 - val_mse: 1.1919e-05 - val_mae: 0.0029
Epoch 46/100
1025/1025 [==============================] - 0s 151us/sample - loss: 1.1153e-05 - mse: 1.1153e-05 - mae: 0.0028 - val_loss: 1.4575e-05 - val_mse: 1.4575e-05 - val_mae: 0.0032
Epoch 47/100
1025/1025 [==============================] - 0s 140us/sample - loss: 1.1300e-05 - mse: 1.1300e-05 - mae: 0.0028 - val_loss: 2.2497e-05 - val_mse: 2.2497e-05 - val_mae: 0.0042
Epoch 48/100
1025/1025 [==============================] - 0s 130us/sample - loss: 1.1257e-05 - mse: 1.1257e-05 - mae: 0.0028 - val_loss: 1.1057e-05 - val_mse: 1.1057e-05 - val_mae: 0.0028
Epoch 49/100
1025/1025 [==============================] - 0s 130us/sample - loss: 1.0582e-05 - mse: 1.0582e-05 - mae: 0.0027 - val_loss: 1.2633e-05 - val_mse: 1.2633e-05 - val_mae: 0.0029
Epoch 50/100
1025/1025 [==============================] - 0s

1025/1025 [==============================] - 0s 148us/sample - loss: 6.7467e-06 - mse: 6.7467e-06 - mae: 0.0022 - val_loss: 1.3354e-05 - val_mse: 1.3354e-05 - val_mae: 0.0029
Epoch 89/100
1025/1025 [==============================] - 0s 222us/sample - loss: 6.8821e-06 - mse: 6.8821e-06 - mae: 0.0022 - val_loss: 1.0504e-05 - val_mse: 1.0504e-05 - val_mae: 0.0026
Epoch 90/100
1025/1025 [==============================] - 0s 207us/sample - loss: 6.6195e-06 - mse: 6.6195e-06 - mae: 0.0021 - val_loss: 8.5944e-06 - val_mse: 8.5944e-06 - val_mae: 0.0024
Epoch 91/100
1025/1025 [==============================] - 0s 209us/sample - loss: 6.4801e-06 - mse: 6.4801e-06 - mae: 0.0021 - val_loss: 7.3387e-06 - val_mse: 7.3387e-06 - val_mae: 0.0022
Epoch 92/100
1025/1025 [==============================] - 0s 190us/sample - loss: 6.3631e-06 - mse: 6.3631e-06 - mae: 0.0021 - val_loss: 1.2783e-05 - val_mse: 1.2783e-05 - val_mae: 0.0029
Epoch 93/100
1025/1025 [==============================] - 0s 170us/sample

In [201]:
fig = go.Figure()
plot_history(fig, history)
fig.show()

In [202]:
fig = go.Figure()
plot_error(fig, model, X_test, y_test)
fig.show()

In [203]:
tf.random.set_seed(2020)

class MyModel(keras.Model):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(4, activation="tanh")
        self.hidden2 = keras.layers.Dense(4, activation="tanh")
        self.hidden3 = keras.layers.Dense(4, activation="tanh")        
        self.out = keras.layers.Dense(1, activation="linear")

    def call(self, input_):
        hidden1 = self.hidden1(input_)
        hidden2 = self.hidden2(hidden1)
        hidden3 = self.hidden3(hidden2)        
        concat = layers.Concatenate()([hidden1, hidden2, hidden3])
        return self.out(concat)

model = MyModel()

In [204]:
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse','mae'])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Train on 1025 samples, validate on 257 samples
Epoch 1/100
1025/1025 [==============================] - 1s 921us/sample - loss: 0.0995 - mse: 0.0995 - mae: 0.1852 - val_loss: 0.0270 - val_mse: 0.0270 - val_mae: 0.1296
Epoch 2/100
1025/1025 [==============================] - 0s 316us/sample - loss: 0.0128 - mse: 0.0128 - mae: 0.0995 - val_loss: 0.0116 - val_mse: 0.0116 - val_mae: 0.0931
Epoch 3/100
1025/1025 [==============================] - 0s 231us/sample - loss: 0.0093 - mse: 0.0093 - mae: 0.0851 - val_loss: 0.0145 - val_mse: 0.0145 - val_mae: 0.1056
Epoch 4/100
1025/1025 [==============================] - 0s 282us/sample - loss: 0.0064 - mse: 0.0064 - mae: 0.0702 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0675
Epoch 5/100
1025/1025 [==============================] - 0s 266us/sample - loss: 0.0038 - mse: 0.0038 - mae: 0.0540 - val_loss: 0.0032 - val_mse: 0.0032 - val_mae: 0.0485
Epoch 6/100
1025/1025 [==============================] - 0s 257us/sample - loss: 0.0021 - mse: 0.0

Epoch 45/100
1025/1025 [==============================] - 0s 227us/sample - loss: 1.1216e-05 - mse: 1.1216e-05 - mae: 0.0028 - val_loss: 1.1919e-05 - val_mse: 1.1919e-05 - val_mae: 0.0029
Epoch 46/100
1025/1025 [==============================] - 0s 177us/sample - loss: 1.1153e-05 - mse: 1.1153e-05 - mae: 0.0028 - val_loss: 1.4575e-05 - val_mse: 1.4575e-05 - val_mae: 0.0032
Epoch 47/100
1025/1025 [==============================] - 0s 156us/sample - loss: 1.1300e-05 - mse: 1.1300e-05 - mae: 0.0028 - val_loss: 2.2497e-05 - val_mse: 2.2497e-05 - val_mae: 0.0042
Epoch 48/100
1025/1025 [==============================] - 0s 149us/sample - loss: 1.1257e-05 - mse: 1.1257e-05 - mae: 0.0028 - val_loss: 1.1057e-05 - val_mse: 1.1057e-05 - val_mae: 0.0028
Epoch 49/100
1025/1025 [==============================] - 0s 161us/sample - loss: 1.0582e-05 - mse: 1.0582e-05 - mae: 0.0027 - val_loss: 1.2633e-05 - val_mse: 1.2633e-05 - val_mae: 0.0029
Epoch 50/100
1025/1025 [==============================] - 0s

1025/1025 [==============================] - 0s 224us/sample - loss: 6.7467e-06 - mse: 6.7467e-06 - mae: 0.0022 - val_loss: 1.3354e-05 - val_mse: 1.3354e-05 - val_mae: 0.0029
Epoch 89/100
1025/1025 [==============================] - 0s 233us/sample - loss: 6.8821e-06 - mse: 6.8821e-06 - mae: 0.0022 - val_loss: 1.0504e-05 - val_mse: 1.0504e-05 - val_mae: 0.0026
Epoch 90/100
1025/1025 [==============================] - 0s 171us/sample - loss: 6.6195e-06 - mse: 6.6195e-06 - mae: 0.0021 - val_loss: 8.5944e-06 - val_mse: 8.5944e-06 - val_mae: 0.0024
Epoch 91/100
1025/1025 [==============================] - 0s 171us/sample - loss: 6.4801e-06 - mse: 6.4801e-06 - mae: 0.0021 - val_loss: 7.3387e-06 - val_mse: 7.3387e-06 - val_mae: 0.0022
Epoch 92/100
1025/1025 [==============================] - 0s 142us/sample - loss: 6.3631e-06 - mse: 6.3631e-06 - mae: 0.0021 - val_loss: 1.2783e-05 - val_mse: 1.2783e-05 - val_mae: 0.0029
Epoch 93/100
1025/1025 [==============================] - 0s 145us/sample

In [205]:
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             multiple                  8         
_________________________________________________________________
dense_61 (Dense)             multiple                  20        
_________________________________________________________________
dense_62 (Dense)             multiple                  20        
_________________________________________________________________
dense_63 (Dense)             multiple                  13        
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [206]:
fig = go.Figure()
plot_history(fig, history)
fig.show()

In [207]:
fig = go.Figure()
plot_error(fig, model, X_test, y_test)
fig.show()

In [208]:
def multi_input(X):
    return np.transpose( np.vstack( (2 * X**2 - 1, 8 * X**4 - 8 * X**2 + 1 )))

In [209]:
tf.random.set_seed(2020)

X_train = np.linspace(0,1,256)
y_train = fun(X_train)

X_valid = np.linspace(0,1,256)
y_valid = fun(X_valid)

X_test = np.linspace(0,1,128)
y_test = fun(X_test)

input_ = layers.Input(shape=2)
hidden1 = layers.Dense(4, activation="tanh")(input_)
hidden2 = layers.Dense(4, activation="tanh")(hidden1)
hidden3 = layers.Dense(4, activation="tanh")(hidden2)
concat = layers.Concatenate()([input_, hidden3])
output = layers.Dense(1, activation="linear")(hidden3)
model = Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_64 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_65 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_66 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_67 (Dense)             (None, 1)                 5         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [210]:
model.compile(loss='mse',
              optimizer=sgd,
              metrics=['mse','mae'])

X_train_3 = multi_input(X_train)
X_valid_3 = multi_input(X_valid)

history = model.fit(X_train_3, y_train, epochs=n_epochs,
                    validation_data=(X_valid_3, y_valid))

Train on 256 samples, validate on 256 samples
Epoch 1/100
256/256 [==============================] - 1s 5ms/sample - loss: 0.1298 - mse: 0.1298 - mae: 0.2244 - val_loss: 0.0085 - val_mse: 0.0085 - val_mae: 0.0805
Epoch 2/100
256/256 [==============================] - 0s 597us/sample - loss: 0.0043 - mse: 0.0043 - mae: 0.0569 - val_loss: 0.0020 - val_mse: 0.0020 - val_mae: 0.0388
Epoch 3/100
256/256 [==============================] - 0s 558us/sample - loss: 8.6683e-04 - mse: 8.6683e-04 - mae: 0.0251 - val_loss: 3.0065e-04 - val_mse: 3.0065e-04 - val_mae: 0.0155
Epoch 4/100
256/256 [==============================] - 0s 497us/sample - loss: 1.6227e-04 - mse: 1.6227e-04 - mae: 0.0106 - val_loss: 6.3690e-05 - val_mse: 6.3690e-05 - val_mae: 0.0062
Epoch 5/100
256/256 [==============================] - 0s 539us/sample - loss: 4.3549e-05 - mse: 4.3549e-05 - mae: 0.0050 - val_loss: 3.4255e-05 - val_mse: 3.4255e-05 - val_mae: 0.0051
Epoch 6/100
256/256 [==============================] - 0s 428us

256/256 [==============================] - 0s 220us/sample - loss: 1.1576e-05 - mse: 1.1576e-05 - mae: 0.0028 - val_loss: 1.0745e-05 - val_mse: 1.0745e-05 - val_mae: 0.0027
Epoch 46/100
256/256 [==============================] - 0s 243us/sample - loss: 1.1270e-05 - mse: 1.1270e-05 - mae: 0.0027 - val_loss: 1.0317e-05 - val_mse: 1.0317e-05 - val_mae: 0.0026
Epoch 47/100
256/256 [==============================] - 0s 237us/sample - loss: 1.0363e-05 - mse: 1.0363e-05 - mae: 0.0026 - val_loss: 1.0119e-05 - val_mse: 1.0119e-05 - val_mae: 0.0026
Epoch 48/100
256/256 [==============================] - 0s 217us/sample - loss: 1.0750e-05 - mse: 1.0750e-05 - mae: 0.0026 - val_loss: 1.0954e-05 - val_mse: 1.0954e-05 - val_mae: 0.0027
Epoch 49/100
256/256 [==============================] - 0s 208us/sample - loss: 1.0437e-05 - mse: 1.0437e-05 - mae: 0.0027 - val_loss: 1.1157e-05 - val_mse: 1.1157e-05 - val_mae: 0.0027
Epoch 50/100
256/256 [==============================] - 0s 207us/sample - loss: 1.0

256/256 [==============================] - 0s 359us/sample - loss: 6.9382e-06 - mse: 6.9382e-06 - mae: 0.0022 - val_loss: 6.4866e-06 - val_mse: 6.4866e-06 - val_mae: 0.0022
Epoch 90/100
256/256 [==============================] - 0s 454us/sample - loss: 6.7483e-06 - mse: 6.7483e-06 - mae: 0.0022 - val_loss: 7.3525e-06 - val_mse: 7.3525e-06 - val_mae: 0.0023
Epoch 91/100
256/256 [==============================] - 0s 488us/sample - loss: 6.6815e-06 - mse: 6.6815e-06 - mae: 0.0022 - val_loss: 6.3845e-06 - val_mse: 6.3845e-06 - val_mae: 0.0022
Epoch 92/100
256/256 [==============================] - 0s 363us/sample - loss: 6.4480e-06 - mse: 6.4480e-06 - mae: 0.0022 - val_loss: 6.7999e-06 - val_mse: 6.7999e-06 - val_mae: 0.0022
Epoch 93/100
256/256 [==============================] - 0s 313us/sample - loss: 6.7538e-06 - mse: 6.7538e-06 - mae: 0.0022 - val_loss: 6.5452e-06 - val_mse: 6.5452e-06 - val_mae: 0.0022
Epoch 94/100
256/256 [==============================] - 0s 371us/sample - loss: 7.0

In [211]:
fig = go.Figure()
plot_history(fig, history)
fig.show()

In [212]:
fig = go.Figure()
y_pred = np.array( model.predict( multi_input(X_test) ) ).flatten()
fig.add_trace(go.Scatter(x = X_test, y = abs(y_test-y_pred)))
fig.update_layout(width=600, yaxis_type="log")
fig.show()